# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np
from itertools import combinations

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    Yes_num = 0
    No_num = 0
    for i in df[label]:
        if i == 'yes':
            Yes_num += 1
        else:
            No_num += 1

    gini = 1 - (((Yes_num / (Yes_num + No_num))**2 + (No_num / (Yes_num + No_num))**2))
    return gini

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
def get_binary_split(df, attribute):
    result = []
    temp = []

    kkk = set(list(combinations(df[attribute], 1))) #조합 중복제거하고 만들기

    for i in kkk:
        temp.append((list(i)[0])) # 리스트 형변환

    x = len(temp)
    for index in range(1 << x):
        result.append([temp[j] for j in range(x) if (index & (1 << j))]) # 부분조합 만들기

    return result[1:-1]

In [6]:
get_binary_split(pd_data, "age")

[['youth'],
 ['senior'],
 ['youth', 'senior'],
 ['middle_aged'],
 ['youth', 'middle_aged'],
 ['senior', 'middle_aged']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    feature = get_binary_split(df, attribute)

    for context in feature:
        context_num = 0 # 해당 속성 갯수
        not_context_num = 0 # 해당 속성 제외 갯수

        context_yes = 0 # 해당 속성 yes 갯수
        context_no = 0 # 해당 속성 no 갯수

        not_context_yes = 0 # 해당 속성 제외 yes 갯수
        not_context_no = 0 # 해당 속성 제외 no 갯수

        context_df = df[[attribute, label]] # 필요변수만 가지고 새로운 데이터프레임 생성

        for i in context_df.values:
            if i[0] in context and i[1] == 'yes':
                context_num += 1
                context_yes += 1

            elif i[0] in context and i[1] == 'no':
                context_num += 1
                context_no += 1

            elif i[0] not in context and i[1] == 'yes':
                not_context_num += 1
                not_context_yes += 1
            else:
                not_context_num += 1
                not_context_no += 1

        s = ",".join(context) # 딕셔너리에 넣을 키값 만들기, 리스트 형식을 문자열로

        # 아래는 수식계산입니다.
        result[s] = (context_num / (context_num + not_context_num)) * (1 - (
                (context_yes / (context_yes + context_no)) ** 2 + (context_no / (context_yes + context_no)) ** 2)) + \
                             (not_context_num / (context_num + not_context_num)) * (1 - (
                (not_context_yes / (not_context_yes + not_context_no)) ** 2 + (not_context_no / (not_context_yes + not_context_no)) ** 2))

    return result

In [8]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'senior': 0.45714285714285713,
 'youth,senior': 0.35714285714285715,
 'middle_aged': 0.35714285714285715,
 'youth,middle_aged': 0.45714285714285713,
 'senior,middle_aged': 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [9]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - youth,senior : 0.35714285714285715


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [10]:
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['high', 'medium'],
 ['low'],
 ['high', 'low'],
 ['medium', 'low']]

In [11]:
print(get_attribute_gini_index(pd_data, "age", "class_buys_computer"))
print(get_attribute_gini_index(pd_data, "income", "class_buys_computer"))
print(get_attribute_gini_index(pd_data, "student", "class_buys_computer"))
print(get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer"))

{'youth': 0.3936507936507937, 'senior': 0.45714285714285713, 'youth,senior': 0.35714285714285715, 'middle_aged': 0.35714285714285715, 'youth,middle_aged': 0.45714285714285713, 'senior,middle_aged': 0.3936507936507937}
{'high': 0.4428571428571429, 'medium': 0.4583333333333333, 'high,medium': 0.45, 'low': 0.45, 'high,low': 0.4583333333333333, 'medium,low': 0.4428571428571429}
{'yes': 0.3673469387755103, 'no': 0.3673469387755103}
{'excellent': 0.42857142857142855, 'fair': 0.42857142857142855}


여기서 age 변수의 지니변수가 다른변수와 비교하였을 때 가장 낮음

In [12]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - youth,senior : 0.35714285714285715


'age' 칼럼을 middle_age 와 그 외 속성값으로 분리하겠습니다.

In [13]:
sub1_df = pd_data[pd_data['age'] == 'middle_aged']
temp_df = pd_data[pd_data['age'] == 'youth']
temp2_df = pd_data[pd_data['age'] == 'senior']
sub2_df = pd.concat([temp_df, temp2_df])

In [14]:
sub1_df

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [15]:
sub2_df

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
10,youth,medium,yes,excellent,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
9,senior,medium,yes,fair,yes
13,senior,medium,no,excellent,no


middle_aged 부터 진행해보겠습니다.

In [17]:
print(get_attribute_gini_index(sub1_df, "age", "class_buys_computer"))
print(get_attribute_gini_index(sub1_df, "income", "class_buys_computer"))
print(get_attribute_gini_index(sub1_df, "student", "class_buys_computer"))
print(get_attribute_gini_index(sub1_df, "credit_rating", "class_buys_computer"))

{}
{'medium': 0.0, 'high': 0.0, 'medium,high': 0.0, 'low': 0.0, 'medium,low': 0.0, 'high,low': 0.0}
{'yes': 0.0, 'no': 0.0}
{'excellent': 0.0, 'fair': 0.0}


타겟 값이 전부 yes 여서 0이 결과가 전부 0으로 나왔습니다. 다음 데이터프레임에 대해서 해보겠습니다.

In [18]:
print(get_attribute_gini_index(sub2_df, "age", "class_buys_computer"))
print(get_attribute_gini_index(sub2_df, "income", "class_buys_computer"))
print(get_attribute_gini_index(sub2_df, "student", "class_buys_computer"))
print(get_attribute_gini_index(sub2_df, "credit_rating", "class_buys_computer"))

{'youth': 0.48, 'senior': 0.48}
{'high': 0.375, 'medium': 0.48, 'high,medium': 0.4761904761904763, 'low': 0.4761904761904763, 'high,low': 0.48, 'medium,low': 0.375}
{'yes': 0.31999999999999984, 'no': 0.31999999999999984}
{'excellent': 0.4166666666666667, 'fair': 0.4166666666666667}


In [ ]:
student의 변수의 지니변수가 가장 낮음을 알 수 있습니다.

In [19]:
my_dict = get_attribute_gini_index(sub2_df, "student", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - yes : 0.31999999999999984
